In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from matplotlib.colors import Normalize, to_hex
import branca.colormap as cm
from IPython.display import display, HTML


from google.colab import drive
drive.mount('/content/drive')

# Load data
file_path= "/content/drive/MyDrive/FINALUHIDATA.csv"
income_visual=pd.read_csv(file_path)


longitude_col = 'longitude'
latitude_col = 'latitude'
income_col = 'Median Income_LST'

norm = Normalize(vmin=income_visual[income_col].min(), vmax=income_visual[income_col].max())
cmap = cm.LinearColormap(['yellow', 'red'], vmin=income_visual[income_col].min(), vmax=income_visual[income_col].max())

geometry = [Point(xy) for xy in zip(income_visual[longitude_col], income_visual[latitude_col])]
geo_data = gpd.GeoDataFrame(income_visual, geometry=geometry)

# Create a base map
m = folium.Map(location=[geo_data[latitude_col].mean(), geo_data[longitude_col].mean()], zoom_start=10)

# Add points to the map with color based on income
for idx, row in geo_data.iterrows():
    folium.CircleMarker(
        [row[latitude_col], row[longitude_col]],
        radius=2.5,
        color=None,
        fill=True,
        fill_color=to_hex(cmap(row[income_col])),
        fill_opacity=0.7,
        popup=f"Income: {row[income_col]}"
    ).add_to(m)

centered_color_bar = folium.Element("""
    <div style="position: fixed; bottom: 50px; left: 50%; transform: translateX(-50%); z-index: 1000; background-color: transparent; padding: 10px; border-radius: 5px;">
        <p style="text-align: center; margin-bottom: 5px;"><strong>Income</strong></p>
        {}
    </div>
""".format(cmap._repr_html_()))

m.get_root().html.add_child(centered_color_bar)

# Save the map to an HTML file
m.save('income_map_heat.html')

# Display the map
display(m)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
